In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/That Fake

/content/drive/.shortcut-targets-by-id/1kA6OIpsKA1jCcdU-bMhxs2O89QudlW2K/That Fake


In [ ]:
import os
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1kA6OIpsKA1jCcdU-bMhxs2O89QudlW2K/That Fake'

In [ ]:
#Importing all the required libraries
!pip3 install face_recognition
import json
import glob
import numpy as np
import cv2
import copy
import os
import random
import time
import sys
import seaborn as sn
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
import torch
import torchvision
from torch import nn
from torchvision import models
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
#Get the average frame count of the videos to crop 
list_video_files=[]
list_video_files =  glob.glob('/content/drive/MyDrive/That Fake/deepfake-detection/deep-fake-dataset/YouTube-real/*.mp4')

frame_count = []
for video_file in list_video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    list_video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [450, 415, 422, 340, 312, 471, 462, 474, 468, 468, 362, 303, 421, 397, 280, 455, 483, 460, 467, 398, 357, 350, 481, 354, 474, 293, 353, 553, 382, 445, 450, 462, 451, 469, 722, 461, 514, 459, 506, 461, 504, 698, 488, 375, 573, 445, 442, 450, 455, 448, 490, 519, 447, 475, 526, 470, 469, 319, 477, 432, 436, 353, 499, 465, 483, 605, 521, 468, 282, 451, 349, 391, 478, 516, 467, 474, 510, 460, 410, 435, 465, 442, 474, 495, 480, 456, 480, 462, 459, 309, 468, 463, 453, 455, 460, 611, 424, 413, 446, 469, 384, 456, 324, 478, 483, 523, 448, 468, 480, 428, 468, 466, 512, 471, 470, 533, 455, 477, 471, 464, 363, 496, 456, 422, 470, 493, 471, 447, 442, 461, 464, 454, 467, 480, 517, 499, 461, 462, 451, 425, 499, 451, 467, 445, 462, 463, 481, 473, 463, 472, 510, 463, 461, 295, 505, 458, 455, 467, 459, 381, 456, 383, 472, 464, 450, 459, 471, 476, 470, 367, 464, 468, 417, 468, 534, 463, 466, 507, 474, 478, 478, 429, 493, 457, 474, 476, 486, 432, 357, 508, 478, 475, 471, 474, 475, 492, 386, 474, 47

In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path) 
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image

def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
create_face_videos(list_video_files,'/content/drive/MyDrive/That Fake/Celeb_REAL_Face_only_data')

In [ ]:
#to load preprocessod video to memory
list_video_files =  glob.glob('/content/drive/MyDrive/That Fake/Celeb_REAL_Face_only_data/*.mp4')
list_video_files = glob.glob('/content/drive/MyDrive/That Fake/deepfake-detection/deep-fake-dataset/Celeb-Youtube-fake/*.mp4')
list_video_files += glob.glob('/content/drive/MyDrive/That Fake/deepfake-detection/deep-fake-dataset/Celeb-real/*.mp4')
list_video_files += glob.glob('/content/drive/MyDrive/That Fake/deepfake-detection/deep-fake-dataset/YouTube-real/*.mp4')
random.shuffle(list_video_files)
frame_count = []
for video_file in list_video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<100):
    list_video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames are  [330, 439, 630, 359, 312, 372, 353, 301, 446, 455, 339, 292, 361, 256, 307, 486, 274, 350, 326, 434, 412, 314, 482, 457, 334, 349, 423, 311, 234, 230, 294, 409, 418, 483, 335, 262, 323, 477, 349, 449, 335, 320, 434, 333, 249, 172, 377, 475, 447, 451, 234, 225, 478, 325, 331, 309, 450, 311, 201, 333, 516, 570, 450, 311, 349, 467, 337, 310, 460, 451, 303, 359, 359, 316, 459, 322, 342, 399, 247, 336, 468, 335, 353, 314, 353, 456, 473, 338, 303, 456, 338, 334, 337, 740, 324, 318, 442, 369, 492, 454, 449, 447, 228, 458, 438, 332, 247, 274, 423, 485, 303, 320, 319, 311, 451, 467, 319, 454, 317, 451, 460, 328, 610, 322, 487, 333, 327, 319, 429, 315, 295, 298, 465, 320, 467, 342, 475, 319, 458, 480, 461, 425, 321, 343, 476, 460, 611, 376, 320, 308, 438, 321, 579, 320, 472, 338, 235, 370, 464, 326, 570, 456, 310, 473, 323, 480, 448, 467, 340, 481, 475, 306, 317, 464, 497, 321, 322, 466, 420, 364, 302, 306, 306, 516, 283, 461, 497, 454, 321, 342, 338, 361, 463, 328, 322, 280, 465, 29

In [ ]:
#To retrieve the final list of video files from the csv file
filename = open('/content/drive/MyDrive/That Fake/video_csv_2.csv', 'r')
file = csv.DictReader(filename)
list_video_files = []

for col in file:
	list_video_files.append(col['File'])

print(list_video_files)


In [ ]:
# Loading the video name and labels from csv

class video_dataset(Dataset):
    def __init__(self,video_names,labels,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.labels = labels
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        temp_video = video_path.split('/')[-1]
        label = self.labels.iloc[(labels.loc[labels['id'] == temp_video].index.values[0]),1]
        if(label == 'FAKE'):
          label = 0
        if(label == 'REAL'):
          label = 1
        for i,frame in enumerate(self.frame_extract(video_path)):
          frames.append(self.transform(frame))
          if(len(frames) == self.count):
            break
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames,label
    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path) 
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image
#To plot the image
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1,2,0)
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
    image = image*255.0
    plt.imshow(image.astype(int))
    plt.show()

In [ ]:
#To count the number of fake and real videos
def number_of_real_and_fake_videos(data_list):
  header_list = ['id','label']
  lab = pd.read_csv('/content/drive/MyDrive/That Fake/Gobal_metadata.csv',names=header_list)
  fake = 0
  real = 0
  for i in data_list:
    temp_video = i.split('/')[-1]
    label = lab.iloc[(labels.loc[labels['id'] == temp_video].index.values[0]),1]
    if(label == 'FAKE'):
      fake+=1
    if(label == 'REAL'):
      real+=1
  return real,fake

In [ ]:
# To load the labels and video in data loader


header_list = ['id','label']
labels = pd.read_csv('/content/drive/MyDrive/That Fake/Gobal_metadata.csv',names=header_list)
train_videos = list_video_files[:int(0.8*len(list_video_files))]
valid_videos = list_video_files[int(0.8*len(list_video_files)):]
print("train : " , len(train_videos))
print("test : " , len(valid_videos))

print("TRAIN: ", "Real:",number_of_real_and_fake_videos(train_videos)[0]," Fake:",number_of_real_and_fake_videos(train_videos)[1])
print("TEST: ", "Real:",number_of_real_and_fake_videos(valid_videos)[0]," Fake:",number_of_real_and_fake_videos(valid_videos)[1])


im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
train_data = video_dataset(train_videos,labels,sequence_length = 10,transform = train_transforms)

val_data = video_dataset(valid_videos,labels,sequence_length = 10,transform = train_transforms)
train_loader = DataLoader(train_data,batch_size = 4,shuffle = True,num_workers = 4)
valid_loader = DataLoader(val_data,batch_size = 4,shuffle = True,num_workers = 4)
image,label = train_data[0]
im_plot(image[0,:,:,:])

In [ ]:

class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) 
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [ ]:
model = Model(2).cuda()
#model = pickle.load(open('finalized_model.sav', 'rb')).cuda()
a,b = model(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.cuda.FloatTensor))

In [ ]:

def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer):
    model.train()
    losses = AverageMeter()
    accuracies = AverageMeter()
    t = []
    for i, (inputs, targets) in enumerate(data_loader):
        if torch.cuda.is_available():
            targets = targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        _,outputs = model(inputs)
        loss  = criterion(outputs,targets.type(torch.cuda.LongTensor))
        acc = calculate_accuracy(outputs, targets.type(torch.cuda.LongTensor))
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    losses.avg,
                    accuracies.avg))
        
    torch.save(model.state_dict(),'/content/drive/MyDrive/That Fake/checkpoint.pt')
    return losses.avg,accuracies.avg
def test(epoch,model, data_loader ,criterion):
    print('Testing')
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    pred = []
    true = []
    count = 0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if torch.cuda.is_available():
                targets = targets.cuda().type(torch.cuda.FloatTensor)
                inputs = inputs.cuda()
            _,outputs = model(inputs)
            loss = torch.mean(criterion(outputs, targets.type(torch.cuda.LongTensor)))
            acc = calculate_accuracy(outputs,targets.type(torch.cuda.LongTensor))
            _,p = torch.max(outputs,1) 
            true += (targets.type(torch.cuda.LongTensor)).detach().cpu().numpy().reshape(len(targets)).tolist()
            pred += p.detach().cpu().numpy().reshape(len(p)).tolist()
            losses.update(loss.item(), inputs.size(0))
            accuracies.update(acc, inputs.size(0))
            sys.stdout.write(
                    "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                    % (
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg
                        )
                    )
        print('\nAccuracy {}'.format(accuracies.avg))
    return true,pred,losses.avg,accuracies.avg
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
def calculate_accuracy(outputs, targets):
    batch_size = targets.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.float().sum().item()
    return 100* n_correct_elems / batch_size

In [ ]:

def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4)
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()
    calculated_acc = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+ cm[1][1])
    print("Calculated Accuracy",calculated_acc*100)

In [ ]:
def plot_loss(train_loss_avg,test_loss_avg,num_epochs):
  loss_train = train_loss_avg
  loss_val = test_loss_avg
  print(num_epochs)
  epochs = range(1,num_epochs+1)
  plt.plot(epochs, loss_train, 'g', label='Training loss')
  plt.plot(epochs, loss_val, 'b', label='validation loss')
  plt.title('Training and Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
def plot_accuracy(train_accuracy,test_accuracy,num_epochs):
  loss_train = train_accuracy
  loss_val = test_accuracy
  epochs = range(1,num_epochs+1)
  plt.plot(epochs, loss_train, 'g', label='Training accuracy')
  plt.plot(epochs, loss_val, 'b', label='validation accuracy')
  plt.title('Training and Validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

lr = 1e-5 
num_epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr= lr,weight_decay = 1e-5)
criterion = nn.CrossEntropyLoss().cuda()
train_loss_avg =[]
train_accuracy = []
test_loss_avg = []
test_accuracy = []
for epoch in range(1,num_epochs+1):
    l, acc = train_epoch(epoch,num_epochs,train_loader,model,criterion,optimizer)
    train_loss_avg.append(l)
    train_accuracy.append(acc)
    true,pred,tl,t_acc = test(epoch,model,valid_loader,criterion)
    test_loss_avg.append(tl)
    test_accuracy.append(t_acc)
    if(epoch%2==0):
      filename = 'OG_Pretrained_Test_model_epoch_20+'+str(epoch)+'.sav'
      pickle.dump(model, open(filename, 'wb'))
    
plot_loss(train_loss_avg,test_loss_avg,len(train_loss_avg))
plot_accuracy(train_accuracy,test_accuracy,len(train_accuracy))
print(confusion_matrix(true,pred))
print_confusion_matrix(true,pred)